Import required modules

In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import json
import patient_data
import cnn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy


2024-12-02 04:16:12.009604: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-02 04:16:12.025942: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733112972.044788   60329 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733112972.050514   60329 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 04:16:12.070430: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Access the folder path for the cancer and the non-cancer images

In [2]:
all_paths = json.loads(open("./paths.json").read())

personal_path = all_paths['personal_path']
non_cancerous_path = personal_path + all_paths['non_cancerous_path']
cancerous_path = personal_path + all_paths['cancerous_path']

Load in all the DICOM files and preprocess/label images

In [3]:
# Using the patient_data data structure, load in all the patient data and save it in a dictionary with the folder name as the key
def load_all_patients(path, add_label = False):
    patients = {}
    folder = os.listdir(path)
    for name in folder:
        patients[name] = patient_data.Patient(os.path.join(path, name))
        if add_label:
            if patients[name].segpath == None:
                print(name, "was not processed correctly")
                patients.pop(name)
            else:
                patients[name].label_imgs()
    return patients

# nc_patients = load_all_patients(non_cancerous_path)
c_patients = load_all_patients(cancerous_path, True)


.ipynb_checkpoints was not processed correctly


Setting up train/test data

In [4]:
# # not sure if we need this
# # create a list for the merged data
# x = []
# y = []

# create a list for only the cancerous dataset data
x_c = []
y_c = []
# # create a list for only the non-cancerous dataset data
# x_nc = []
# y_nc = []

for patient in c_patients.values():
    for i, img in enumerate(patient.ct.data.values()):
        x_c.append(img)
        y_c.append(patient.labels[i])
        # # not sure if we need this
        # x.append(img)
        # y.append(patient.labels[i])

x_c, y_c = shuffle(x_c, y_c)
c_patients = None

# for patient in nc_patients.values():
#     for i, img in enumerate(patient.ct.images):
#         x_nc.append(img)
#         y_nc.append(patient.labels[i])
#         # # not sure if we need this
#         # x.append(img)
#         # y.append(patient.labels[i])

# # not sure if we need this
# # Shuffle the merged data
# combined = list(zip(x, y))
# np.random.shuffle(combined)
# x2, y2 = zip(*combined)

# def generate_train_test():
#     # to ensure equal distribution of non-cancer to cancer data, split the data before merging it
#     x_train, x_test, y_train, y_test = train_test_split(x_c, y_c, test_size=0.2, random_state=42)

#     # x_train_add, x_test_add, y_train_add, y_test_add = train_test_split(x_nc, y_nc, test_size=0.2, random_state=42)
#     # x_train.extend(x_train_add) 
#     # x_test.extend(x_test_add) 
#     # y_train.extend(y_train_add) 
#     # y_test.extend(y_test_add) 

#     # Convert lists to arrays
#     x_train = np.array(x_train)/255
#     x_test = np.array(x_test)/255
#     y_train = np.array(y_train)
#     y_test = np.array(y_test)

#     return x_train, x_test, y_train, y_test, x_train[0]

# x_train, x_test, y_train, y_test, test = generate_train_test()

In [5]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print('Num GPUs Available: ', len(physical_devices))
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


In [6]:
import gc

class MyCustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect
        tf.keras.backend.clear_session()

In [7]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from tensorflow.keras import backend as K
# Define K-Fold Cross-Validation
n_splits = 3
kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
x_data = np.array(x_c)/255  # Normalize the images
y_data = np.array(y_c)
# Model training and evaluation loop
fold_results = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(x_c, y_c)):
    print(f"\nTraining fold {fold + 1}/{n_splits}")
    
    # Split data
    x_train, x_val = x_data[train_idx], x_data[val_idx]
    y_train, y_val = y_data[train_idx], y_data[val_idx]
    print('point a passed')

    # Build the model
    model = Sequential([
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(512, 512, 1)),
        MaxPool2D(pool_size=(2, 2), strides=2),
        Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
        MaxPool2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(units=1, activation='sigmoid')
    ])
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    print('point b passed')


    from tensorflow.keras.callbacks import EarlyStopping

    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Train the model
    model.fit(
        x=x_train,
        y=y_train,
        validation_data=(x_val, y_val),
        batch_size=10,
        epochs=20,
        callbacks=[MyCustomCallback()],
        verbose=1
    )
    print("Model architecture built")
    print('point c passed')
    # Evaluate the model
    predictions = (model.predict(x_val) > 0.5).astype("int32")
    report = classification_report(y_val, predictions, output_dict=True)
    print(classification_report(y_val, predictions))
    
    # Save fold results
    fold_results.append(report)
    K.clear_session()


# Aggregate results
avg_accuracy = np.mean([fold['accuracy'] for fold in fold_results])
print(f"\nAverage Accuracy Across {n_splits} Folds: {avg_accuracy:.4f}")


Training fold 1/3
point a passed


/home/jovyan/conda-envs/venv2/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1733113072.105727   60329 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43598 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:ca:00.0, compute capability: 8.6


point b passed
Epoch 1/20


I0000 00:00:1733113105.035784   60733 service.cc:148] XLA service 0x7f58c80046a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733113105.035843   60733 service.cc:156]   StreamExecutor device (0): NVIDIA A40, Compute Capability 8.6
2024-12-02 04:18:25.115212: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1733113105.254119   60733 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-12-02 04:18:25.347016: W external/local_xla/xla/service/gpu/nvptx_compiler.cc:930] The NVIDIA driver's CUDA version is 12.2 which is older than the PTX compiler version 12.5.82. Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


   7/2037 ━━━━━━━━━━━━━━━━━━━━ 44s 22ms/step - accuracy: 0.5516 - loss: 0.8358

...

2037/2037 ━━━━━━━━━━━━━━━━━━━━ 70s 33ms/step - accuracy: 0.8540 - loss: 0.3578 - val_accuracy: 0.8863 - val_loss: 0.2377
Epoch 2/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 50s 24ms/step - accuracy: 0.9017 - loss: 0.2204 - val_accuracy: 0.9284 - val_loss: 0.1740
Epoch 3/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 48s 24ms/step - accuracy: 0.9327 - loss: 0.1567 - val_accuracy: 0.9437 - val_loss: 0.1399
Epoch 4/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 49s 24ms/step - accuracy: 0.9560 - loss: 0.1108 - val_accuracy: 0.9485 - val_loss: 0.1256
Epoch 5/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 50s 24ms/step - accuracy: 0.9649 - loss: 0.0922 - val_accuracy: 0.9620 - val_loss: 0.1067
Epoch 6/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 50s 24ms/step - accuracy: 0.9777 - loss: 0.0656 - val_accuracy: 0.9641 - val_loss: 0.0934
Epoch 7/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 50s 24ms/step - accuracy: 0.9856 - loss: 0.0450 - val_accuracy: 0.9671 - val_loss: 0.0876
Epoch 8/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 51s 25ms/step - accuracy: 0.9899 - loss: 0.03

/home/jovyan/conda-envs/venv2/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 65s 31ms/step - accuracy: 0.8489 - loss: 0.3499 - val_accuracy: 0.8631 - val_loss: 0.2947
Epoch 2/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 50s 25ms/step - accuracy: 0.9105 - loss: 0.2045 - val_accuracy: 0.9113 - val_loss: 0.1976
Epoch 3/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 50s 24ms/step - accuracy: 0.9403 - loss: 0.1481 - val_accuracy: 0.9356 - val_loss: 0.1447
Epoch 4/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 51s 25ms/step - accuracy: 0.9604 - loss: 0.1085 - val_accuracy: 0.9488 - val_loss: 0.1209
Epoch 5/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 50s 24ms/step - accuracy: 0.9711 - loss: 0.0787 - val_accuracy: 0.9554 - val_loss: 0.1054
Epoch 6/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 50s 25ms/step - accuracy: 0.9818 - loss: 0.0558 - val_accuracy: 0.9498 - val_loss: 0.1275
Epoch 7/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 51s 25ms/step - accuracy: 0.9860 - loss: 0.0454 - val_accuracy: 0.9530 - val_loss: 0.1141
Epoch 8/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 50s 24ms/step - accuracy: 0.9873 -

/home/jovyan/conda-envs/venv2/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 70s 33ms/step - accuracy: 0.8562 - loss: 0.3482 - val_accuracy: 0.8954 - val_loss: 0.2310
Epoch 2/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 49s 24ms/step - accuracy: 0.9107 - loss: 0.2101 - val_accuracy: 0.9302 - val_loss: 0.1685
Epoch 3/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 49s 24ms/step - accuracy: 0.9400 - loss: 0.1431 - val_accuracy: 0.9402 - val_loss: 0.1425
Epoch 4/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 49s 24ms/step - accuracy: 0.9582 - loss: 0.1088 - val_accuracy: 0.9256 - val_loss: 0.1620
Epoch 5/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 50s 25ms/step - accuracy: 0.9687 - loss: 0.0832 - val_accuracy: 0.9595 - val_loss: 0.1056
Epoch 6/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 49s 24ms/step - accuracy: 0.9789 - loss: 0.0615 - val_accuracy: 0.9644 - val_loss: 0.0975
Epoch 7/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 49s 24ms/step - accuracy: 0.9854 - loss: 0.0450 - val_accuracy: 0.9694 - val_loss: 0.0914
Epoch 8/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 50s 25ms/step - accuracy: 0.9900 -

In [2]:
model.summary()

NameError: name 'model' is not defined

In [1]:
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
def getgradcam_heatmap_fixed(model, img_array, last_conv_layer):
    grad_model = Model(inputs=model.input, outputs=[last_conv_layer.output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(conv_outputs * pooled_grads, axis=-1)
    heatmap = tf.maximum(heatmap, 0) / tf.reduce_max(heatmap)
    return heatmap.numpy()

def compute_saliency_map_fixed(model, img_array):
    with tf.GradientTape() as tape:
        tape.watch(img_array)
        predictions = model(img_array)
        pred_index = tf.argmax(predictions[0])
        loss = predictions[:, pred_index]
    grads = tape.gradient(loss, img_array)
    saliency = tf.reduce_max(tf.abs(grads), axis=-1).numpy()
    return saliency[0]

test_image_array = x_c  # Replace 'processed_image' with your actual preprocessed variable

if isinstance(test_image_array, list):
    test_image_array = np.stack(test_image_array)  # Convert list of tensors to a single 4D tensor
if len(test_image_array.shape) == 3:  # If missing batch dimension
    test_image_array = np.expand_dims(test_image_array, axis=0)
if len(test_image_array.shape) == 2:  # If missing channel dimension
    test_image_array = np.expand_dims(test_image_array, axis=-1)

test_image_array = tf.convert_to_tensor(test_image_array, dtype=tf.float32)

last_conv_layer_name = None
for layer in reversed(model.layers):
    if 'conv' in layer.name:
        last_conv_layer_name = layer.name
        break

if last_conv_layer_name is None:
    raise ValueError("No convolutional layer found in the model.")

last_conv_layer = model.get_layer(last_conv_layer_name)

heatmap = get_gradcam_heatmap_fixed(model, test_image_array, last_conv_layer)
plt.imshow(heatmap, cmap="jet")
plt.axis("off")
plt.title("Grad-CAM Heatmap")
plt.show()

saliency = compute_saliency_map_fixed(model, test_image_array)
plt.imshow(saliency, cmap="viridis")
plt.axis("off")
plt.title("Saliency Map")
plt.show()

2024-12-02 05:56:22.154797: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-02 05:56:22.781406: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733118983.010492   96129 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733118983.073945   96129 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 05:56:23.668822: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

NameError: name 'x_c' is not defined

In [ ]:
print(len(y_test))
for i, j in enumerate(y_test):
    k = predictions[i][0]
    if j != k:
        print(j, k)

Train and test CNN model

In [ ]:

# num_tests = 1
# cnns = []
# for i in range(num_tests):
# cnns.append(cnn.CNN(x_train, x_test, y_train, y_test))

Cross validation and bootstrapping

In [ ]:
# print(cnns[0].test_acc)